In [4]:
import json
import pandas as pd

with open('../data/triviaqa-unfiltered/unfiltered-web-dev.json','r') as f:
    data = json.loads(f.read()) # Flatten data
df = pd.json_normalize(data, record_path =['Data'])
df = df[['Question', 'QuestionId', 'QuestionSource', 'Answer.Aliases', 'Answer.Value']]
df

,Question,QuestionId,QuestionSource,Answer.Aliases,Answer.Value
0,Who was the man behind The Chipmunks?,tc_2,http://www.triviacountry.com/,[David Seville],David Seville
1,What star sign is Jamie Lee Curtis?,tc_13,http://www.triviacountry.com/,"[Scorpio, Skorpio, Scorpio (disambiguation)]",Scorpio
2,Which Lloyd Webber musical premiered in the US...,tc_33,http://www.triviacountry.com/,"[Sunset Blvd, West Sunset Boulevard, Sunset Bo...",Sunset Boulevard
3,Who was the next British Prime Minister after ...,tc_40,http://www.triviacountry.com/,"[Sir Henry Campbell-Bannerman, Campbell-Banner...",Campbell-Bannerman
4,Who had a 70s No 1 hit with Kiss You All Over?,tc_49,http://www.triviacountry.com/,"[Internal exile, Exiles, Transported for life,...",Exile
...,...,...,...,...,...
11308,Who tried to steal Christmas from the town of ...,qg_4615,https://quizguy.wordpress.com/,"[Grinch, The grinch, The Grinch]",The Grinch
11309,"According to the proverb, what comes but once ...",qg_4620,https://quizguy.wordpress.com/,"[Orthodox Christmas, Holiday decorations, Chir...",Christmas
11310,What is the name of the parson mentioned in th...,qg_4628,https://quizguy.wordpress.com/,"[Orangeish blacks, Light brown, Orangeish blac...",Brown
11311,"According to a popular Christmas song, who was...",qg_4629,https://quizguy.wordpress.com/,"[Frosty The Snowman, Frosty and Rudolph's Chri...",Frosty the Snowman.


In [7]:
df[(df['Question'].str.startswith("Which country"))]

,Question,QuestionId,QuestionSource,Answer.Aliases,Answer.Value
73,Which country does the airline TAAG come from?,tc_704,http://www.triviacountry.com/,"[Angola, Angloa, Etymology of Angola, Republic...",Angola
252,Which country is known to its people as Suomen...,tc_2506,http://www.triviacountry.com/,"[FinlanD, FINLAND, Suomen tasavalta, Suomen Ta...",Finland
532,Which country grows the largest number of mang...,qz_2841,https://www.quiz-zone.co.uk/,"[ഭാരത മഹാരാജ്യം, هندستانڀارت،, भारतीय गणराज्या...",India
704,Which country does the drink Cinzano come from?,qz_5332,https://www.quiz-zone.co.uk/,"[Environment of Italy, Italiën, Subdivisions o...",Italy
964,Which country was invaded by Iraq in 1990?,qb_1435,http://www.quizballs.com/,"[ISO 3166-1:KW, Kuwayt, State of Kuwait, KUW, ...",Kuwait
1107,Which country will host the 2014 FIFA World Cup?,qb_3042,http://www.quizballs.com/,"[Brazil, Etymology of Brazil, Federal Republic...",Brazil
1108,Which country has the internet domain .de?,qb_3045,http://www.quizballs.com/,"[United States of Germany, Tyskland, Nimska, B...",Germany
1369,Which country was formerly known by the Greeks...,qb_5539,http://www.quizballs.com/,"[Asia/Bahrain, مملكة البحرين, Bahrein, Bahrayn...",Bahrain
1553,Which country invaded the Falkland Islands in ...,qb_7476,http://www.quizballs.com/,"[Arxintina, Argintina, Argentyna, آرژانتین, Re...",Argentina
1854,Which country is known as The Hashemite Kingdom?,dpql_49,https://derbyshirepubquizleague.wordpress.com/,"[Jordan (country), Al-Urdunn, أردنّ, JOrdan, U...",Jordan
